In [ ]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.inspection import permutation_importance
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Bước 1: Đọc tệp CSV
file_path = "../dataset/data.csv"  # Cập nhật đường dẫn tới tệp CSV của bạn
data = pd.read_csv(file_path)

data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

In [ ]:
# Bước 2: Bỏ cột 'IsHoliday' và xác định biến độc lập (X) và biến phụ thuộc (y)
X = data.drop(columns=['Weekly_Sales', 'IsHoliday'])
y = data['Weekly_Sales']


In [ ]:
# Bước 3: Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Khởi tạo và huấn luyện mô hình SVR với các tham số tùy chỉnh
model = SVR(kernel='rbf', C=1.0, epsilon=0.1)
model.fit(X_train, y_train)

In [ ]:
# Bước 4: Lấy tầm quan trọng của các đặc trưng
result = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42)

coef_df = pd.DataFrame({
    'Variable': X.columns,
    'Importance': result.importances_mean
}).sort_values(by='Importance', ascending=False)

# In tầm quan trọng của từng biến
print(coef_df)

In [ ]:
# Bước 5: Dự báo doanh số cho Cửa hàng 1 và Bộ phận 1
store_1_dept_1 = data[(data['Store'] == 1) & (data['Dept'] == 1)]
X_store_dept = store_1_dept_1.drop(columns=['Weekly_Sales', 'IsHoliday'])

# Dự báo doanh số cho Cửa hàng 1 và Bộ phận 1
store_dept_sales_pred = model.predict(X_store_dept)

# Doanh số thực tế cho Cửa hàng 1 và Bộ phận 1
store_dept_sales_actual = store_1_dept_1['Weekly_Sales']


In [ ]:
# Bước 6: Vẽ đồ thị so sánh doanh số thực tế và dự báo
plt.figure(figsize=(12, 6))
plt.plot(store_1_dept_1.index, store_dept_sales_actual, label='Doanh số thực tế', color='blue', marker='o', linestyle='-')
plt.plot(store_1_dept_1.index, store_dept_sales_pred, label='Doanh số dự báo', color='orange', marker='x', linestyle='--')
plt.xlabel('Ngày')
plt.ylabel('Doanh số hàng tuần')
plt.title('So sánh Doanh số Thực tế và Dự báo cho Cửa hàng 1 và Bộ phận 1')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Bước 7: Dự báo hai tuần tiếp theo
last_date = store_1_dept_1.index.max()

next_two_weeks = pd.date_range(start=last_date + pd.Timedelta(days=7), periods=2, freq='W')

new_data = pd.DataFrame({
    'Store': [1, 1],
    'Dept': [1, 1],
    'Size': [store_1_dept_1['Size'].mean(), store_1_dept_1['Size'].mean()],
    'Temperature': [60.0, 58.0],
    'Fuel_Price': [3.5, 3.6],
    'MarkDown1': [0.0, 0.0],
    'MarkDown2': [0.0, 0.0],
    'MarkDown3': [0.0, 0.0],
    'MarkDown4': [0.0, 0.0],
    'MarkDown5': [0.0, 0.0],
    'CPI': [211.0, 211.0],
    'Unemployment': [8.0, 8.0]
}, index=next_two_weeks)

# Dự báo doanh số cho hai tuần tiếp theo
predicted_sales_next_2_weeks = model.predict(new_data)

new_data['Predicted_Weekly_Sales'] = predicted_sales_next_2_weeks

# In doanh số dự báo cho hai tuần tiếp theo
print("Doanh số dự báo cho 2 tuần tiếp theo:")
print(new_data[['Predicted_Weekly_Sales']])


In [ ]:
# Bước 8: Vẽ đồ thị doanh số dự báo cho hai tuần tiếp theo
plt.figure(figsize=(12, 6))
plt.plot(store_1_dept_1.index, store_dept_sales_actual, label='Doanh số thực tế', color='blue', marker='o', linestyle='-')
plt.plot(store_1_dept_1.index, store_dept_sales_pred, label='Doanh số dự báo', color='orange', marker='x', linestyle='--')
plt.plot(new_data.index, new_data['Predicted_Weekly_Sales'], label='Dự báo 2 tuần tiếp theo', color='green', marker='o', linestyle='--')
plt.xlabel('Ngày')
plt.ylabel('Doanh số hàng tuần')
plt.title('So sánh Doanh số Thực tế, Dự báo và Dự báo 2 Tuần Tiếp theo cho Cửa hàng 1 và Bộ phận 1')
plt.legend()
plt.grid()
plt.show()